In [1]:
library(tidyverse);library(RMySQL)
conn <- dbConnect(
  MySQL(),
  user = 'root', 
  password = 'skt1fa!ker', 
  host = 'localhost',
  client.flag = CLIENT_MULTI_RESULTS
)
dbGetQuery(conn, "USE market_db")

-- Attaching packages ------------------------------------------------------------------------------- tidyverse 1.3.1 --

v ggplot2 3.3.5.9000     v purrr   0.3.4     
v tibble  3.1.6          v dplyr   1.0.8     
v tidyr   1.2.0          v stringr 1.4.0     
v readr   2.1.2          v forcats 0.5.1     

-- Conflicts ---------------------------------------------------------------------------------- tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()

Loading required package: DBI



<0 x 0 matrix>

### 동적 SQL

### ```PREPARE ~ FROM``` : SQL문을 준비만 해놓음
### ```EXECUTE ~ USING``` : PREPARE한 것을 대입하여 실행
### ```DEALLOCATE PREPARE``` : EXECUTE한 후 문장을 해제 (초기화)

- **PREPARE은 ‘SELECT’문을 실행하지 않고 mySQL이라는 이름으로 준비**
- **EXECUTE로 mySQL에 저장된 ‘SELECT’문을 실행할 때 USING으로 ‘?’에 @limit 변수 대입**

In [6]:
dbGetQuery(conn, "SET @limit = 3")
dbGetQuery(conn, "PREPARE mySQL FROM 
                  'SELECT * FROM world.city ORDER BY Population DESC LIMIT ?';")
dbGetQuery(conn, "EXECUTE mySQL USING @limit; ")

ID,Name,CountryCode,District,Population
<int>,<chr>,<chr>,<chr>,<int>
1024,Mumbai (Bombay),IND,Maharashtra,10500000
2331,Seoul,KOR,Seoul,9981619
206,S횄짙o Paulo,BRA,S횄짙o Paulo,9968485


#### **

In [11]:
# - gate_table 테이블 생성 (id는 정수형, 자동입력 / entry_time은 날자 시간형 )
dbGetQuery(conn, "CREATE TABLE gate_table (
                  id INT AUTO_INCREMENT PRIMARY KEY, 
                  entry_time DATETIME);")

# - curDate를 현재 시간으로 설정
dbGetQuery(conn, "SET @curDate = CURRENT_TIMESTAMP();")

# - PREPARE로 ‘INSERT’문을 myQuery라는 이름으로 저장
dbGetQuery(conn, "PREPARE myQuery FROM 'INSERT INTO gate_table VALUES(NULL, ?)';")

# - EXECUTE로 ‘INSERT’문의 ‘?’에 @curDate가 대입, 
# 즉 EXECUTE문이 실행된 시점의 날짜시간이 gate_table테이블에 저장
dbGetQuery(conn, "EXECUTE myQuery USING @curDate;")

# - DEALLOCATE PREFARE로 문장 해제(myQuery문 초기화)
dbGetQuery(conn, "DEALLOCATE PREPARE myQuery;")

dbGetQuery(conn, "SELECT * FROM gate_table;") 

<0 x 0 matrix>

<0 x 0 matrix>

<0 x 0 matrix>

<0 x 0 matrix>

<0 x 0 matrix>

id,entry_time
<int>,<chr>
1,2022-06-27 02:21:58


### 동적 SQL 스토어드 프로시저

**tableName에 테이블 이름을 입력하면, 입력된 테이블을 호출하는 스토어드 프로시저**

```SQL
DELIMITER $$
CREATE PROCEDURE dynamic_proc(
	In tableName VARCHAR(20)
)
BEGIN
	SET @sqlQuery = CONCAT('SELECT * FROM ', tableName);
    PREPARE myQuery FROM @sqlQuery;
    EXECUTE myQuery;
    DEALLOCATE PREPARE myQuery;
END $$
DELIMITER ;
```

In [16]:
dbGetQuery(conn, "CALL dynamic_proc('SSOL.heart');") 

,Age,SEX,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Olepeak,ST_Slope,HeartDisease
,<int>,<chr>,<chr>,<int>,<int>,<int>,<chr>,<int>,<chr>,<dbl>,<chr>,<int>
1,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
2,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
3,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
4,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
5,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


# 

### 스토어드 함수 : 직접 만들어서 사용하는 함수

## 
### 스토어드 함수 생성 권한 부여
### ```SET GLOBAL log_bin_trust_function_creators = 1;```

### 스토어드 함수 생성

### ```CREATE FUNCTION ~ RETURNS```
```SQL
DELIMITER $$
CREATE FUNCTION {스토어드_함수_이름} (매개변수)
RETURNS {반환형식}
BEGIN
~ 
END $$
DELIMITER ;
```

### **시간형 데이터의 년 차이를 계산하는 스토어드 함수 생성**

```SQL
DELIMITER $$
CREATE FUNCTION calcYearFunc(dYear INT)
    RETURNS INT
BEGIN
    DECLARE runYear INT;
    SET runYear = YEAR(CURDATE()) - dYear;
    RETURN runYear;
END $$
DELIMITER ; 
```

In [20]:
dbGetQuery(conn, "SELECT calcYearFunc(2007) INTO @debut2007;")
dbGetQuery(conn, "SELECT calcYearFunc(2013) INTO @debut2013;")

<0 x 0 matrix>

<0 x 0 matrix>

In [21]:
dbGetQuery(conn, "SELECT @debut2007 - @debut2013 AS 'DATE DIFF between 2007 and 2013';")

DATE DIFF between 2007 and 2013
<dbl>
6


**스토어드 함수를 통해 member 테이블의 활동 햇수 계산**

In [19]:
dbGetQuery(conn, "SELECT mem_id, debut_date FROM member;")

mem_id,debut_date
<chr>,<chr>
APN,2011-02-10
BLK,2016-08-08
GRL,2007-08-02
ITZ,2019-02-12
MMU,2014-06-19
OMY,2015-04-21
RED,2014-08-01
SPC,2016-02-25
TWC,2015-10-19


In [22]:
dbGetQuery(conn, "SELECT mem_id, calcYearFunc(YEAR(debut_date)) AS 'Act Period' 
                  FROM member; ")

mem_id,Act Period
<chr>,<int>
APN,11
BLK,6
GRL,15
ITZ,3
MMU,8
OMY,7
RED,8
SPC,6
TWC,7


### ```DROP FUNCTION``` : 스토어드 함수 제거

In [23]:
dbGetQuery(conn, "DROP FUNCTION calcYearFunc;")

<0 x 0 matrix>

# 

## 커서 : 모든 행을 한 행씩 처리할 때 사용
- 변수 준비 -> 커서 선언 -> 반복 조건 선언 -> 커서 열기 -> 데이터 가져오기 -> 데이터 처리하기 -> 커서 닫기

### 1. 변수 준비

### 2. 커서 선언
### ```DECLARE {커서} CURSOR FOR```

```SQL
DECLARE 커서_이름 CURSOR FOR
	SELECT문 ~ FROM 테이블 ;
```
- SELECT문에 해당하는 커서 생성

### 3. 반복조건 선언
### ```DECLARE CONTINUE HANDLER``` 
### ```FOR NOT FOUND SET {SQL문}```

- **DECLARE CONTINUE HANDLER로 반복 조건 준비**
- **반복할 행이 없을 때 FOR NOT FOUND SET으로 SQL문 실행**
- 예) : 행의 끝을 판단하기 위해 변수 endOfROW를 준비 => TRUE값인지 체크하는 방식

### 4. 커서 열기
### ```OPEN {커서};```

### 5. 행 반복

### ```LOOP ~ FETCH {커서} INTO {변수}; ```
### ```{조건문} END LOOP;```
```SQL
cursor_loop: LOOP

   FETCH {커서} INTO {준비한_변수} ; 

   IF {조건문} THEN
	LEAVE cursor_loop ;
   END iF ; 

END LOOP cursor_loop ;
```
- FETCH문을 통해 한 행씩 커서 실행
- IF 조건문을 통해 반복 종료 조건 설정
- END LOOP를 통해 커서 종료


### 6. 커서 닫기
### ```CLOSE {커서};```

#  

**회원의 평균 인원수를 계산하기 위한 스토어드 프로시저 생성**

```SQL
DELIMITER $$
CREATE PROCEDURE cursor_proc()
BEGIN
    DECLARE memNumber INT; -- 지역변수 
    DECLARE cnt INT DEFAULT 0; -- 지역변수 
    DECLARE totNumber INT DEFAULT 0; -- 지역변수 
    DECLARE endOfRow BOOLEAN DEFAULT FALSE; -- 행의 끝 여부(기본을 FALSE)

    DECLARE memberCuror CURSOR FOR
        SELECT mem_number FROM member;
 -- 커서 선언 (member테이블의 mem_number 호출)

    DECLARE CONTINUE HANDLER
        FOR NOT FOUND SET endOfRow = TRUE;
 -- 반복 조건을 준비하는 예약어
 -- 행의 끝이면 endOfRow 변수에 TRUE를 대입 

    OPEN memberCuror;  -- 커서 열기
    cursor_loop: LOOP
        FETCH  memberCuror INTO memNumber; 
 -- 커서로 선언한 select문의 조회 내용을 memNumber로 지정

        IF endOfRow THEN 
            LEAVE cursor_loop ; -- endOfRow가 TRUE면 반복 중단
        END IF ; 
        SET cnt = cnt + 1;
        SET totNumber = totNumber + memNumber;        
    END LOOP cursor_loop;

    SELECT (totNumber/cnt) AS 'Average number of member';
    CLOSE memberCuror; 

END $$
DELIMITER ;
```

In [27]:
dbGetQuery(conn, "CALL cursor_proc();")

Warning message in .local(conn, statement, ...):
"Decimal MySQL column 0 imported as numeric"


Average number of member
<dbl>
6.6


# 

### 트리거 : (Trigger) 하나의 SQL문이 실행되면, 자동으로 다른 SQL문 실행
- 예 ; 테이블에서 하나 데이터가 삭제 -> 삭제가 되기전 자동으로 다른 테이블에 저장
- 테이블에 DML문 (INSERT, UPDATE, DELETE) 실행 => 트리거가 자동으로 작동
- **테이블에 입력/수정/삭제되는 정보를 백업하는 용도 활용 가능** (데이터의 무결성)

### ```CREATE TRIGGER {트리거}``` : 트리거 생성
```SQL
DELIMITER $$
CREATE TRIGGER {트리거}
   AFTER {DML문}
   ON {테이블}
   FOR EACH ROW
BEGIN
   {SQL문} ; 
   -- DML문이 실행되면, 자동으로 테이블의 모든 열에, BEGIN ~ END 사이의 SQL문이 실행 
END $$
DELIMITER ;
```


### ```DELETE TRIGGER {트리거}``` : 트리거 삭제

### 트리거 활용 
**siger 테이블과, 백업 테이블인 backup_siger 생성**

In [30]:
dbGetQuery(conn, "CREATE TABLE singer (SELECT mem_id, mem_number FROM member);")
dbGetQuery(conn, "CREATE TABLE backup_singer(
                  mem_id  CHAR(8) NOT NULL , 
                  mem_number INT NOT NULL, 
                  modType  CHAR(2), 
                  modDate  DATE, 
                  modUser  VARCHAR(30)) ;")

<0 x 0 matrix>

<0 x 0 matrix>

In [12]:
dbGetQuery(conn, "SELECT * FROM singer;")
dbGetQuery(conn, "SELECT * FROM backup_singer;")

mem_id,mem_number
<chr>,<int>
APN,6
BLK,4
GRL,8
ITZ,5
MMU,4
OMY,7
RED,4
SPC,13
TWC,9


mem_id,mem_number,modType,modDate,modUser
<chr>,<int>,<chr>,<chr>,<chr>


#### 변경(UPDATE)이 발생했을 때, 변경된 이력과 데이터를 backup_singer 테이블에 저장하는 트리거 생성

```SQL
DELIMITER $$
CREATE TRIGGER singer_updateTrg 
    AFTER UPDATE -- 변경 후에 작동하도록 지정
    ON singer -- 트리거를 부착할 테이블
    FOR EACH ROW 
BEGIN
    INSERT INTO backup_singer VALUES (OLD.mem_id, OLD.mem_number, 'Updated', CURDATE(), CURRENT_USER());
END $$ 
DELIMITER ; 
```

#### 삭제(DELETE)가 발생했을 때, 삭제된 이력과 데이터를 backup_singer 테이블에 저장하는 트리거 생성
- **TRUNATE문에는 DELETE 트리거가 작동하지 않음** 

```SQL
DELIMITER $$
CREATE TRIGGER singer_deleteTrg 
    AFTER DELETE -- 삭제 후에 작동하도록 지정
    ON singer 
    FOR EACH ROW 
BEGIN
    INSERT INTO backup_singer VALUES (OLD.mem_id, OLD.mem_number, 'Deleted', CURDATE(), CURRENT_USER());
END $$ 
DELIMITER ; 
```

### SHOW TRIGGERS FROM {데이터베이스} : DB내 생성된 트리거 확인

In [6]:
dbGetQuery(conn, "SHOW TRIGGERS FROM market_db;")

Warning message in .local(conn, statement, ...):
"unrecognized MySQL field type 7 in column 5 imported as character"


Trigger,Event,Table,Statement,Timing,Created,sql_mode,Definer,character_set_client,collation_connection,Database Collation
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
singer_updateTrg,UPDATE,singer,"BEGIN INSERT INTO backup_singer VALUES (OLD.mem_id, OLD.mem_number, 'Updated', CURDATE(), CURRENT_USER()); END",AFTER,2022-06-27 03:57:03.11,"STRICT_TRANS_TABLES,NO_ENGINE_SUBSTITUTION",root@localhost,utf8mb4,utf8mb4_0900_ai_ci,utf8mb4_0900_ai_ci
singer_deleteTrg,DELETE,singer,"BEGIN INSERT INTO backup_singer VALUES (OLD.mem_id, OLD.mem_number, 'Deleted', CURDATE(), CURRENT_USER()); END",AFTER,2022-06-27 03:57:05.09,"STRICT_TRANS_TABLES,NO_ENGINE_SUBSTITUTION",root@localhost,utf8mb4,utf8mb4_0900_ai_ci,utf8mb4_0900_ai_ci


#### UPDATE와 DELETE 적용후, 이력과 데이터가 저장된 backup_singer 테이블 확인

In [13]:
dbGetQuery(conn, "UPDATE singer SET mem_number = 5 
                  WHERE mem_id = 'APN';")
dbGetQuery(conn, "DELETE FROM singer WHERE mem_number >= 7;")
dbGetQuery(conn, "SELECT * FROM backup_singer;")

<0 x 0 matrix>

<0 x 0 matrix>

mem_id,mem_number,modType,modDate,modUser
<chr>,<int>,<chr>,<chr>,<chr>
APN,6,Updated,2022-06-27,root@localhost
GRL,8,Deleted,2022-06-27,root@localhost
OMY,7,Deleted,2022-06-27,root@localhost
SPC,13,Deleted,2022-06-27,root@localhost
TWC,9,Deleted,2022-06-27,root@localhost


# 

### OLD / NEW : INSERT, UPDATE, DELETE작업이 수행되면 임시로 사용되는 MySQL 내장 테이블
<img src = "https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FcCWZjL%2FbtqRQTu5agJ%2FJmCfLJAr59HpkN6Ali4GQ1%2Fimg.png">